# Lab 7: Object-Oriented Python

## Overview

After last time's lab, which covered the basics of OOP, we'll continue playing with classes today, writing a fair chunk of code and building several classes to solve a variety of problems.

Recall these definitions to guide you through this lab:

- An *object* has identity
- A *name* is a reference to an object
- A *namespace* is an associative mapping from names to objects
- An *attribute* is any name following a dot ('.')

## Part #1 Concrete Object Oriented Programming

Enough monkeying around! Let's have you work on a number of objects that you may one day encounter in your programmer life.

### Exercise #1 Not-so-SimpleGraph

In this part, you'll build the implementation for a `SimpleGraph` class in Python.

You will need to define a `Vertex` class, an `Edge` class, and a `SimpleGraph` class. The specification is as follows:

(1) A `Vertex` has attributes:

* `name`, a string representing the label of the vertex.
* `edges`, a set representing edges outbound from this vertex to its neighbors

A new Vertex should be initialized with an optional `name`, which defaults to `""`, and should be initialized with an empty edge set.
Also with methods such as `__str__`, `__repr__` to make the print more readable. **You are free to add any other methods that considered useful.**

(2) An `Edge` has attributes:

* `start`, a `Vertex` representing the start point of the edge.
* `end`, a `Vertex` representing the end point of the edge.
* `cost`, a `float` (used for graph algorithms) representing the weight of the edge.
* `visited`, a `bool` (used for graph algorithms) representing whether this edge has been visited before.

Note that for our purposes, an `Edge` is directed: the edge from `v_a` to `v_b` is distinct from the edge from `v_b` to `v_a`. **You can make the direction explict by using arrows such as `-->` in the `__str__` method.**

An `Edge` requires a `start` and `end` vertex in order to be instantiated. `cost` should default to 1, and `visited` should default to `False`, but both should be able to be set via an initializer.

(3) A `SimpleGraph` has attributes

* `verts`, a collection of `Vertex`s (you need to decide the collection type)
* `edges`, a collection of `Edge`s (you need to decide the collection type)

as well as several methods:

* `graph.add_vertex(v)`
* `graph.add_edge(v_1, v_2)`
* `graph.contains_vertex(v)` - return `True` or `False`
* `graph.contains_edge(v_1, v_2)` - return `True` or `False`
* `graph.get_neighbors(v)` - return get a list of incoming neighbors and a list of outgoing neighbors
* `graph.is_empty()` - return `True` or `False`
* `graph.size()` - return number of edges |E|
* `graph.remove_vertex(v)` **tip: first remove the edges attached to the vertex**
* `graph.remove_edge(v_1, v_2)`
* `graph.is_neighbor(v1, v2)` - return `True` or `False`; direction of neighborhood does not matter here
* `graph.is_reachable(v1, v2)` - return `True` or `False`, use any algorithm you like
* `graph.clear_all()`

The actual implementation details are up to you.

*Note: debugging will be significantly easier if you write `__str__` or `__repr__` methods on your custom classes.*

In [56]:
from collections import defaultdict


class Vertex:
    def __init__(self, name="", edges=None):
        self.name = name
        self.edges = set() if edges is None else edges

    def __repr__(self):
        return f"V({self.name})"

    # def __str__(self):
    #     return f"V({self.name})"


class Edge:
    def __init__(self, start: Vertex, end: Vertex, cost=1, visited=False):
        self.start = start
        self.end = end
        self.cost = cost
        self.visited = visited

    def __repr__(self):
        return f"{self.start.name} ---{self.cost}---> {self.end.name}"


class SimpleGraph:
    def __init__(self, verts=None, edges=None):
        # keys are unique identifiers(names)
        # and values are the corresponding `Vertex` objects
        self.verts = {v.name: v for v in (verts or [])}  # ex: {'1': Vertex('1'), ...}
        #  keys are vertices and values are lists of outgoing edges
        #  defaultdict help avoid checks for existence of vertices
        self.edges = defaultdict(list)
        if edges is not None:
            for edge in edges:
                self.edges[edge.start.name].append(
                    (edge.end.name, edge.cost))  # ex: {'1': [('3', 0.2), ('2', 0.7)], '7': [('8', 0.2)]}

    def __repr__(self):
        edges_repr = "\n".join(
            [f"{Vertex(start)} ---{cost}---> {Vertex(end)}," for start, ends in self.edges.items() for end, cost in
             ends])
        return f"---- SimpleGraph ----{{\n{edges_repr}\n}}"

    def __getitem__(self, key):
        return self.verts[key]

    def is_empty(self):
        # the graph with no vertices is an empty graph G=({},{})
        return not bool(self.verts)

    def size(self):
        # the size of  graph is the total number of edges
        return sum(len(edges) for edges in self.edges.values())

    def is_reachable(self, v1, v2):
        """
            Check for reachability means that exists path between any 2 vertices in graph,
            by moving from one vertex to its neighbors
        """
        visited = set()     # track the visited vertices
        st = [v1]
        while st:        # iterate while stack isn't empty
            top = st.pop()
            if top == v2:       # check if the current vertex is target vertex - path reached
                return True
            if top not in visited:      # check is current vertex haven't been visited before
                visited.add(top)        # if no, save to visited
                for neighbor in self.edges[top.name]:       # iterate over neighbors of current vertex
                    if neighbor[0] not in visited:      # if they haven't been visited
                        st.append(self.verts[neighbor[0]])      # add to stack
        return False

    def is_neighbor(self, v1, v2):
        # check if v1 is a neighbor of v2
        for neighbor, _ in self.edges.get(v1.name, []):
            if neighbor == v2.name:
                return True
        # check if v2 is a neighbor of v1
        for neighbor, _ in self.edges.get(v2.name, []):
            if neighbor == v1.name:
                return True
        # if no return False
        return False

    def get_neighbors(self, v):
        out_neighbors = []      # outgoing neighbors of vertex
        in_neighbors = []       # incoming neighbors of vertex

        # iterate over the outgoing edges
        for neighbor, _ in self.edges.get(v.name, []):
            out_neighbors.append(self.verts[neighbor])

        # iterate over all edges
        for key, value in self.edges.items():
            for neighbor, _ in value:
                if neighbor == v.name:      # finding the incoming neighbors
                    in_neighbors.append(self.verts[key])

        return in_neighbors, out_neighbors

    def remove_edge(self, v1, v2):
        """Removes edge  between 2 vertices."""
        if v1.name in self.edges:       # check if v1 has edges
            # remove edge v1--v2
            self.edges[v1.name] = [(neighbor, cost) for neighbor, cost in self.edges[v1.name] if neighbor != v2.name]
        if v2.name in self.edges:       # check if v2 has edges
            # remove edge v2--v1
            self.edges[v2.name] = [(neighbor, cost) for neighbor, cost in self.edges[v2.name] if neighbor != v1.name]

    def remove_vertex(self, vertex):
        """Removes vertex and connected edges."""
        if vertex.name in self.verts:   # check if vertex exists
            # delete the vertex and connected edges
            del self.verts[vertex.name]
            del self.edges[vertex.name]

            for v in self.edges:    # iterate over edges
                # exclude the removed vertex from all other vertices' edge lists
                self.edges[v] = [(neighbor, cost) for neighbor, cost in self.edges[v] if neighbor != vertex.name]

    def add_vertex(self, vertex):
        """Adds vertex to graph."""
        if vertex.name not in self.verts:       # check if vertex not exist in graph
            self.verts[vertex.name] = vertex
            self.edges[vertex.name] = []

    def add_edge(self, v1, v2, cost=1):
        """Adds an edge between two vertices."""
        if v1.name in self.verts and v2.name in self.verts:     # check if both vertices exists in graph
            self.edges[v1.name].append((v2.name, cost))

    def contains_vertex(self, vertex):
        """Checks if a given vertex is present in the graph"""
        return vertex in self.verts.values()

    def contains_edge(self, start_vertex, end_vertex):
        """ Checks if there is an edge between two given vertices"""
        if start_vertex.name not in self.verts or end_vertex.name not in self.verts:    # check if both vertices in graph
            return False
        for edge in self.edges[start_vertex.name]:      # iterate over associated edges
            if edge[0] == end_vertex.name:      # check the matching
                return True
        return False

#### Some example to test your clases on

In [57]:
# Testing code
# graph-like data for testing
rand_graph = [
    ('1', '2', 0.7),
    ('1', '3', 0.2),
    ('1', '4', 0.3),
    ('2', '3', 0.3),
    ('2', '5', 0.4),
    ('3', '5', 0.4),
    ('3', '8', 0.5),
    ('4', '12', 0.2),
    ('5', '6', 0.1),
    ('6', '8', 0.3),
    ('7', '8', 0.2),
    ('7', '13', 0.2),
    ('9', '10', 0.6),
    ('9', '11', 0.4),
    ('9', '12', 0.4),
    ('10', '12', 0.1),
    ('11', '13', 0.5),
]

# construct set of verts and edges with above data
verts = {}
edges = set()
for v1, v2, cost in rand_graph:
    start = Vertex(v1)
    end = Vertex(v2)
    edge = Edge(start, end, cost)
    edges.add(edge)
    if v1 not in verts:
        verts[v1] = start
    if v2 not in verts:
        verts[v2] = end
    if edge not in verts[v1].edges:
        verts[v1].edges.add(edge)

# construct SimpleGraph with set of verts and edges
graph = SimpleGraph(set(verts.values()), edges)

# Some tests
print("===== Basic structure ====")
print(graph)
# Results could be a visualization such as:
# ---- SimpleGraph ----{
# V(1) ---0.3---> V(4),
# V(6) ---0.3---> V(8),
# V(7) ---0.2---> V(13),
# V(10) ---0.1---> V(12),
# V(2) ---0.4---> V(5),
# V(11) ---0.5---> V(13),
# V(2) ---0.3---> V(3),
# V(5) ---0.1---> V(6),
# V(9) ---0.4---> V(12),
# V(1) ---0.2---> V(3),
# V(7) ---0.2---> V(8),
# V(9) ---0.4---> V(11),
# V(3) ---0.4---> V(5),
# V(3) ---0.5---> V(8),
# V(1) ---0.7---> V(2),
# V(9) ---0.6---> V(10),
# V(4) ---0.2---> V(12)}
print(graph.is_empty())
# False
print(graph.size())
# 17
print(set(graph.verts.values()))
# {V(6), V(11), V(12), V(13), V(9), V(2), V(3), V(1), V(10), V(7), V(4), V(5), V(8)}

print("===== Reachability =====")
print(graph.is_reachable(verts["1"], verts["7"])) # False
print(graph.is_reachable(verts["1"], verts["2"])) # True
print(graph.is_reachable(verts["1"], verts["8"])) # True
print("===== Are neighbors? =====")
print(graph.is_neighbor(verts["7"], verts["8"])) #True
print(graph.is_neighbor(verts["8"], verts["7"])) #True
print(graph.is_neighbor(verts["6"], verts["12"])) #False
print("===== All neighbors =====")
print(graph.get_neighbors(verts["1"])) # ([], [V(3), V(2), V(4)])
print(graph.get_neighbors(verts["3"])) # ([V(2), V(1)], [V(5), V(8)])
print("===== Remove edge =====")
graph.remove_edge(Vertex("1"), Vertex("3"))
print(graph.size())
# 16
print(graph.get_neighbors(verts["1"]))
# ([], [V(4), V(2)])
print("===== Remove vertex =====")
graph.remove_vertex(Vertex("2"))
print(graph.size()) #13
print(graph)
# ---- SimpleGraph ----{
# V(1) ---0.3---> V(4),
# V(6) ---0.3---> V(8),
# V(7) ---0.2---> V(13),
# V(10) ---0.1---> V(12),
# V(11) ---0.5---> V(13),
# V(5) ---0.1---> V(6),
# V(9) ---0.4---> V(12),
# V(7) ---0.2---> V(8),
# V(9) ---0.4---> V(11),
# V(3) ---0.4---> V(5),
# V(3) ---0.5---> V(8),
# V(9) ---0.6---> V(10),
# V(4) ---0.2---> V(12)}
# ---------------------
# Test add_vertex
vertex_1 = Vertex("14")
vertex_2 = Vertex("15")
graph.add_vertex(vertex_1)
graph.add_vertex(vertex_2)
print(set(graph.verts.values()))

# Test contains_vertex
print(graph.contains_vertex(vertex_1))  # True
print(graph.contains_vertex(Vertex("3")))  # False

# Test add_edge
vertex_3 = Vertex("3")
graph.add_vertex(vertex_3)
graph.add_edge(vertex_1, vertex_3, 11)
print(graph)

# Test contains_edge
print(graph.contains_edge(vertex_1, vertex_3))  # True
print(graph.contains_edge(vertex_2, vertex_3))  # False

===== Basic structure ====
---- SimpleGraph ----{
V(3) ---0.4---> V(5),
V(3) ---0.5---> V(8),
V(2) ---0.4---> V(5),
V(2) ---0.3---> V(3),
V(9) ---0.4---> V(12),
V(9) ---0.4---> V(11),
V(9) ---0.6---> V(10),
V(1) ---0.2---> V(3),
V(1) ---0.7---> V(2),
V(1) ---0.3---> V(4),
V(7) ---0.2---> V(13),
V(7) ---0.2---> V(8),
V(11) ---0.5---> V(13),
V(10) ---0.1---> V(12),
V(6) ---0.3---> V(8),
V(5) ---0.1---> V(6),
V(4) ---0.2---> V(12),
}
False
17
{V(12), V(5), V(7), V(3), V(13), V(2), V(4), V(9), V(11), V(6), V(8), V(1), V(10)}
===== Reachability =====
False
True
True
===== Are neighbors? =====
True
True
False
===== All neighbors =====
([], [V(3), V(2), V(4)])
([V(2), V(1)], [V(5), V(8)])
===== Remove edge =====
16
([], [V(2), V(4)])
===== Remove vertex =====
13
---- SimpleGraph ----{
V(3) ---0.4---> V(5),
V(3) ---0.5---> V(8),
V(9) ---0.4---> V(12),
V(9) ---0.4---> V(11),
V(9) ---0.6---> V(10),
V(1) ---0.3---> V(4),
V(7) ---0.2---> V(13),
V(7) ---0.2---> V(8),
V(11) ---0.5---> V(13),
V(10) -

### Exercise #6: Polynomial Class

We will write a `Polynomial` class that acts like a number. As a a reminder, a [polynomial](https://en.wikipedia.org/wiki/Polynomial) is a mathematical object that looks like $1 + x + x^2$ or $4 - 10x + x^3$ or $-4 - 2x^{10}$. A mathematical polynomial can be evaluated at a given value of $x$. For example, if $f(x) = 1 + x + x^2$, then $f(5) = 1 + 5 + 5^2 = 1 + 5 + 25 = 31$.

Polynomials are also added componentwise: If $f(x) = 1 + 4x + 4x^3$ and $g(x) = 2 + 3x^2 + 5x^3$, then $(f + g)(x) = (1 + 2) + 4x + 3x^2 + (4 + 5)x^3 = 3 + 4 + 3x^2 + 9x^3$.

Construct a polynomial with variadic coefficients keywords: the zeroth coefficient is given with `c_0`, the square coefficient is given with `c_2`, and so on. For example, `f = Polynomial(c_0=1, c_2=3, c_4=5)` should construct a `Polynomial` representing $1 + 3x^2 + 5x^4$.

You will need to override the addition special method (`__add__`) and the callable special method (`__call__`).

You should be able to emulate the following code:

```Python
f = Polynomial(c_0=1, c_1=5, c_2=10)
g = Polynomial(c_0=1, c_1=3, c_2=5)

print(f(5))  # => Invokes `f.__call__(5)`
print(g(2))  # => Invokes `g.__call__(2)`

h = f + g    # => Invokes `f.__add__(g)`
print(h(3))  # => Invokes `h.__call__(3)`
```

Lastly, implement a method to convert a `Polynomial` to an informal string representation. For example, the polynomial `Polynomial(1, 3, 5)` should be represented by the string `"1 * x^0 + 3 * x^1 + 5 * x^2"`.

**Bonus** (you'll get bonus marks if you implement this correctly): Implement also the special method `__sub__` to subtract two polynomials, and `__mul__` which lets you multiply a polynomial either by a scalar or another polynomial (see sample code below for examples)

In [58]:
# A class to represent polynomials of any degree

class Polynomial:
    def __init__(self, **coeff):
        # **coeff - initialization with variable number of coefficients
        # store the coeff in a dictionary with the power as the key
        self.coefficients = {int(key.split('_')[-1]): value for key, value in coeff.items()}

    def __call__(self, x):
        """Estimate the polynomial with a given value of x"""
        # calculate each term and sum them
        return sum(coeff * (x ** power) for power, coeff in self.coefficients.items())

    def __add__(self, other):
        """Sum of 2 polynomials"""
        # create a new polynomial with the same coeff
        result = Polynomial(**{f"c_{power}": coeff for power, coeff in self.coefficients.items()})
        # iterate over the coeff of the other
        for power, coeff in other.coefficients.items():
            # add the coeff of the same power or create a new entry if it doesn't exist
            result.coefficients[power] = result.coefficients.get(power, 0) + coeff
        return result

    def __sub__(self, other):
        """Subtract two polynomials"""
        # create a new polynomial with the same coeff
        result = Polynomial(**{f"c_{power}": coeff for power, coeff in self.coefficients.items()})
        # iterate over the coeff of the other
        for power, coeff in other.coefficients.items():
            # subtract the coef of the same power or create a new entry if it doesn't exist
            result.coefficients[power] = result.coefficients.get(power, 0) - coeff
        return result

    def __mul__(self, other):
        """Multiply two polynomials or a polynomial and a scalar"""
        # create an empty polynomial
        result = Polynomial()
        # check the type of other: polinomial or scalar
        if isinstance(other, Polynomial):
            # iterate over the coeff of both polynomials
            for power_1, coeff_1 in self.coefficients.items():
                for power_2, coeff_2 in other.coefficients.items():
                    power = power_1 + power_2
                    # ddd the product term to the result or create a new entry if it doesn't exist
                    result.coefficients[power] = result.coefficients.get(power, 0) + coeff_1 * coeff_2
        else:
            for power, coeff in self.coefficients.items():
                # multiply the coefficient by the scalar and store it
                result.coefficients[power] = coeff * other
        return result

    def __str__(self):
        terms = [f"{coeff} * x^{power}" if power > 0 else f"{coeff}" for power, coeff in self.coefficients.items()]
        return " + ".join(terms)

In [59]:
# Testing code
f = Polynomial(c_0=1, c_1=5, c_2=10)
g = Polynomial(c_0=1, c_1=3, c_2=5)
h = Polynomial(c_0=1, c_5=7)

print(f"given f = {f}; f(5) = {f(5)}")
print(f"given g = {g}; g(2) = {g(2)}")
print(f"given h = {h}; g(3) = {g(3)}")
# given f = 1 + 5x + 10x^2; f(5) = 276
# given g = 1 + 3x + 5x^2; g(2) = 27
# given h = 1 + 7x^5; g(3) = 55

print(f"({f}) + ({g}) = {f + g}")
print(f"({f}) + ({h}) = {f + h}")

# Optional: subtraction and multiplication
print(f"({f}) - ({g}) = {f - g}")
print(f"({f})5 = {f * 5}")
print(f"({f})({g}) = {f * g}")
# (1 + 5x + 10x^2) - (1 + 3x + 5x^2) = 2x + 5x^2
# (1 + 5x + 10x^2)5 = 5 + 25x + 50x^2
# (1 + 5x + 10x^2)(1 + 3x + 5x^2) = 1 + 8x + 30x^2 + 55x^3 + 50x^4

given f = 1 + 5 * x^1 + 10 * x^2; f(5) = 276
given g = 1 + 3 * x^1 + 5 * x^2; g(2) = 27
given h = 1 + 7 * x^5; g(3) = 55
(1 + 5 * x^1 + 10 * x^2) + (1 + 3 * x^1 + 5 * x^2) = 2 + 8 * x^1 + 15 * x^2
(1 + 5 * x^1 + 10 * x^2) + (1 + 7 * x^5) = 2 + 5 * x^1 + 10 * x^2 + 7 * x^5
(1 + 5 * x^1 + 10 * x^2) - (1 + 3 * x^1 + 5 * x^2) = 0 + 2 * x^1 + 5 * x^2
(1 + 5 * x^1 + 10 * x^2)5 = 5 + 25 * x^1 + 50 * x^2
(1 + 5 * x^1 + 10 * x^2)(1 + 3 * x^1 + 5 * x^2) = 1 + 8 * x^1 + 30 * x^2 + 55 * x^3 + 50 * x^4


# Submission instructions

You will need to exercises #1 `Not-so-SimpleGraph` and #6 `Polynomial Class` on Arche before **9:59am on Friday, 8th December 2023**. Submit either a `.py` or an `.ipynb` file containing the exercises and name it `td7_firstname_lastname_grpN.py` or `td7_firstname_lastname_grpN.ipynb` accordingly, where `firstname` should be your first name, `lastname` should be your last name, and `N` in `grpN` should be your group number (e.g. Jane Doe, who is in group A1, should name her submission either `td7_jane_doe_grp1.py` or `td7_jane_doe_grp1.ipynb`, depending on whether Jane submitted a Python script or a Jupyter notebook).

To evaluate your submission, we will be looking at the following criteria:

- Does your code run? (So **run** your program at least once before submitting!)
- Does it run correctly? (So **test** your solution with a few different inputs!)
- Is your code well-commented?

> Stolen from CS Stanford. With <3 by @sredmond

> Adapted by tmickus